In [1]:
from pyspark import SparkConf,SparkContext
conf = SparkConf()
conf.set("spark.master","local[3]")
conf.set("spark.app.name","sampleapp")
sc = SparkContext(conf=conf)

<SparkContext master=local[3] appName=sampleapp>

movies1 = sc.textFile("/home/ubuntu/ml-1m/movies.dat")
movies2 = movies1.map(lambda x:x.split("::"))
print(movies2.take(5))

In [7]:
from pyspark.sql import SQLContext

sqlc = SQLContext(sc)

movies1 = sc.textFile("/home/ubuntu/ml-1m/movies.dat")
for i in movies1.take(5):
    print(i)
movies2 = movies1.map(lambda x:x.split("::")) 
print("-----------")
for i in movies2.take(5):
    print(i)
movies3 = movies2. map(lambda x:(int(x[0]),x[1],x[2]))
print("----------------")
for i in movies3.take(5):
    print(i)

#create a dataframe for movies3 rdd

moviesdf1 = movies3.toDF(["movieId","Title","Genres"])
print("------------")
print(moviesdf1.show(10))

1::Toy Story (1995)::Animation|Children's|Comedy
2::Jumanji (1995)::Adventure|Children's|Fantasy
3::Grumpier Old Men (1995)::Comedy|Romance
4::Waiting to Exhale (1995)::Comedy|Drama
5::Father of the Bride Part II (1995)::Comedy
-----------
[u'1', u'Toy Story (1995)', u"Animation|Children's|Comedy"]
[u'2', u'Jumanji (1995)', u"Adventure|Children's|Fantasy"]
[u'3', u'Grumpier Old Men (1995)', u'Comedy|Romance']
[u'4', u'Waiting to Exhale (1995)', u'Comedy|Drama']
[u'5', u'Father of the Bride Part II (1995)', u'Comedy']
----------------
(1, u'Toy Story (1995)', u"Animation|Children's|Comedy")
(2, u'Jumanji (1995)', u"Adventure|Children's|Fantasy")
(3, u'Grumpier Old Men (1995)', u'Comedy|Romance')
(4, u'Waiting to Exhale (1995)', u'Comedy|Drama')
(5, u'Father of the Bride Part II (1995)', u'Comedy')
------------
+-------+--------------------+--------------------+
|movieId|               Title|              Genres|
+-------+--------------------+--------------------+
|      1|    Toy Story 

In [12]:
ratings1 = sc.textFile("/home/ubuntu/ml-1m/ratings.dat") 
ratings2 = ratings1.map(lambda x:x.split("::")) 
ratings3 = ratings2.map(lambda x:(int(x[0]),int(x[1]),int(x[2]),x[3]))
ratingsdf1 = ratings3.toDF(["userId","moviesId","rating","timestamp"])
print(ratingsdf1.show(10))

+------+--------+------+---------+
|userId|moviesId|rating|timestamp|
+------+--------+------+---------+
|     1|    1193|     5|978300760|
|     1|     661|     3|978302109|
|     1|     914|     3|978301968|
|     1|    3408|     4|978300275|
|     1|    2355|     5|978824291|
|     1|    1197|     3|978302268|
|     1|    1287|     5|978302039|
|     1|    2804|     5|978300719|
|     1|     594|     4|978302268|
|     1|     919|     4|978301368|
+------+--------+------+---------+
only showing top 10 rows

None


ratings1 = sc.textFile("/home/ubuntu/ml-1m/ratings.dat")
ratings2 = ratings1.map(lambda x:x.split("::"))
ratings3 = ratings2.map(lambda x:(int(x[0]),int(x[1]),int(x[2]),x[3]))
ratingsdf1 = ratings3.toDF(["userId","moviesId","rating","timestamp"])
print(ratingsdf1.show(10))

# Top ten most viewed movies with their movies Name (Ascending or Descending order)  


In [20]:
from pyspark.sql.functions import col,count

mr1 = moviesdf1.join(ratingsdf1,col("movieId")==col("moviesId"),"inner")
print(mr1.show(10))
mr1.cache()  #preserves the dat of mr1 in memory

+-------+--------------+------+------+--------+------+----------+
|movieId|         Title|Genres|userId|moviesId|rating| timestamp|
+-------+--------------+------+------+--------+------+----------+
|     26|Othello (1995)| Drama|    18|      26|     4| 978157335|
|     26|Othello (1995)| Drama|    69|      26|     4| 977882050|
|     26|Othello (1995)| Drama|   229|      26|     4|1039503573|
|     26|Othello (1995)| Drama|   342|      26|     4| 976338677|
|     26|Othello (1995)| Drama|   524|      26|     3| 976169012|
|     26|Othello (1995)| Drama|   655|      26|     3| 975699243|
|     26|Othello (1995)| Drama|   748|      26|     5| 975463153|
|     26|Othello (1995)| Drama|   881|      26|     3|1013536125|
|     26|Othello (1995)| Drama|   890|      26|     3| 976504835|
|     26|Othello (1995)| Drama|   918|      26|     4| 978241611|
+-------+--------------+------+------+--------+------+----------+
only showing top 10 rows

None


DataFrame[movieId: bigint, Title: string, Genres: string, userId: bigint, moviesId: bigint, rating: bigint, timestamp: string]

In [30]:
from pyspark.sql.functions import *
mr2 = mr1.select(col("Title"),col("userId")).groupBy(col("Title")).agg(count(col("userId")).alias("Views"))
mr3 = mr2.orderBy(desc("Views")).limit(10)
print(mr3.show(20))

+--------------------+-----+
|               Title|Views|
+--------------------+-----+
|American Beauty (...| 3428|
|Star Wars: Episod...| 2991|
|Star Wars: Episod...| 2990|
|Star Wars: Episod...| 2883|
|Jurassic Park (1993)| 2672|
|Saving Private Ry...| 2653|
|Terminator 2: Jud...| 2649|
|  Matrix, The (1999)| 2590|
|Back to the Futur...| 2583|
|Silence of the La...| 2578|
+--------------------+-----+

None


In [31]:
mr1.registerTempTable("table1")
res = sqlc.sql("select Title,count(userId) as views from table1 group by Title order by views desc limit 10 ")
print(res.show())

+--------------------+-----+
|               Title|views|
+--------------------+-----+
|American Beauty (...| 3428|
|Star Wars: Episod...| 2991|
|Star Wars: Episod...| 2990|
|Star Wars: Episod...| 2883|
|Jurassic Park (1993)| 2672|
|Saving Private Ry...| 2653|
|Terminator 2: Jud...| 2649|
|  Matrix, The (1999)| 2590|
|Back to the Futur...| 2583|
|Silence of the La...| 2578|
+--------------------+-----+

None


In [37]:
# Top twenty rated movies (Condition: The movie should be rated/viewed by at least 40 users
res2 = mr1.select("Title","rating","userId").groupBy(col("Title")).agg(count(col("userId")).alias("views"),avg(col("rating")).alias("ratings"))
print(res2.count())
res3 = res2.where('views>=40').orderBy(desc("ratings")).limit(20)
print(res3.count())
print(res3.show())


3706
20
+--------------------+-----+------------------+
|               Title|views|           ratings|
+--------------------+-----+------------------+
|      Sanjuro (1962)|   69| 4.608695652173913|
|Seven Samurai (Th...|  628| 4.560509554140127|
|Shawshank Redempt...| 2227| 4.554557700942973|
|Godfather, The (1...| 2223| 4.524966261808367|
|Close Shave, A (1...|  657|  4.52054794520548|
|Usual Suspects, T...| 1783| 4.517106001121705|
|Schindler's List ...| 2304| 4.510416666666667|
|Wrong Trousers, T...|  882| 4.507936507936508|
|Sunset Blvd. (a.k...|  470| 4.491489361702127|
|Raiders of the Lo...| 2514| 4.477724741447892|
|  Rear Window (1954)| 1050| 4.476190476190476|
|Paths of Glory (1...|  230| 4.473913043478261|
|Star Wars: Episod...| 2991| 4.453694416583082|
|Third Man, The (1...|  480| 4.452083333333333|
|Dr. Strangelove o...| 1367|4.4498902706656915|
|Wallace & Gromit:...|  438| 4.426940639269406|
|To Kill a Mocking...|  928| 4.425646551724138|
|Double Indemnity ...|  551| 4.4

In [42]:
res4 = sqlc.sql("select Title,count(userId) as views ,avg(rating) as ratings from table1 group by Title order by ratings desc").where('views>=40').limit(20)
print(res4.show())

+--------------------+-----+------------------+
|               Title|views|           ratings|
+--------------------+-----+------------------+
|      Sanjuro (1962)|   69| 4.608695652173913|
|Seven Samurai (Th...|  628| 4.560509554140127|
|Shawshank Redempt...| 2227| 4.554557700942973|
|Godfather, The (1...| 2223| 4.524966261808367|
|Close Shave, A (1...|  657|  4.52054794520548|
|Usual Suspects, T...| 1783| 4.517106001121705|
|Schindler's List ...| 2304| 4.510416666666667|
|Wrong Trousers, T...|  882| 4.507936507936508|
|Sunset Blvd. (a.k...|  470| 4.491489361702127|
|Raiders of the Lo...| 2514| 4.477724741447892|
|  Rear Window (1954)| 1050| 4.476190476190476|
|Paths of Glory (1...|  230| 4.473913043478261|
|Star Wars: Episod...| 2991| 4.453694416583082|
|Third Man, The (1...|  480| 4.452083333333333|
|Dr. Strangelove o...| 1367|4.4498902706656915|
|Wallace & Gromit:...|  438| 4.426940639269406|
|To Kill a Mocking...|  928| 4.425646551724138|
|Double Indemnity ...|  551| 4.415607985

In [43]:
res4.write.format("csv").option("header","True").save("/home/ubuntu/sol2.csv")
res.write.format("csv").option("header","True").save("/home/ubuntu/sol1.csv")

movies1 = sc.textFile("/home/ubuntu/ml-1m/movies.dat")
for i in movies1.take(5):
    print(i)
movies2 = movies1.map(lambda x:x.split("::")) 
print("-----------")
for i in movies2.take(5):
    print(i)
movies3 = movies2. map(lambda x:(int(x[0]),x[1],x[2]))
for i in movies3.take(5):
